In [32]:
using Distributions, Statistics

# 1 - Maintenance problem

- Constants : 

In [33]:
# all costs are written in lists as follows [New,good shape,old,broken]
maintenance_cost = 10
repair_costs = [0,15,30,50]
monthly_earnings = [30,20,10,0]
replacement_cost = 70
Transition_matrix_without_maintenance = [
    0 1 0 0;
    0 0.3 0.7 0;
    0 0 0.5 0.5;
    0 0 0 1
]
Transition_matrix_with_maintenance = [
    0 0 0 0; # a new machine cannot be maintained
    0 0.8 0.2 0;
    0 0 0.9 0.1;
    0 0 0 0 # a broken machine cannot be maintained
]
# the next two matrices are just to make the main code easier it essentially just means you cannot repair or replace a new machine 
Transition_matrix_of_repairs = [
    0 0 0 0;
    0 1 0 0;
    0 1 0 0;
    0 1 0 0;
]
Transition_matrix_of_replacement = [
    0 0 0 0;
    1 0 0 0;
    1 0 0 0;
    1 0 0 0;
]
number_of_months = 12 
vector_of_possibilities = [
    Transition_matrix_without_maintenance,
    Transition_matrix_with_maintenance,
    Transition_matrix_of_repairs,
    Transition_matrix_of_replacement
]

4-element Vector{Matrix{Float64}}:
 [0.0 1.0 0.0 0.0; 0.0 0.3 0.7 0.0; 0.0 0.0 0.5 0.5; 0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 0.0; 0.0 0.8 0.2 0.0; 0.0 0.0 0.9 0.1; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0]

- the main problem :

In [34]:
function cost_of_action(state,possibility)
    if possibility == 1 
        return 0
    elseif possibility == 2 
        return 10
    elseif possibility == 3 
        return repair_costs[state]
    else 
        return 70
    end
end

cost_of_action (generic function with 1 method)

In [35]:
decision_matrix = zeros(4,number_of_months)
cost_matrix = zeros(4,number_of_months)
cost_matrix[:,number_of_months] = monthly_earnings
for month ∈ number_of_months-1:-1:1
    for state ∈ 1:4
        cost_of_possibilities = [0.0 0.0 0.0 0.0]
        for possibility ∈ 1:4
           cost_of_possibilities[possibility] = monthly_earnings[state] + (vector_of_possibilities[possibility][state,:]')*cost_matrix[:,month+1] - cost_of_action(state,possibility)
        end
        val, idx = findmax(cost_of_possibilities)
        cost_matrix[state,month] = val
        decision_matrix[state,month] = idx[2]
    end
end
best_cost = cost_matrix[1,1]

110.82

# 2 - Stock management

- constants :

In [62]:
T = 14
price_up = 1 
price_down = 3
storage_cost = 0.1
initial_stock = 10
n = 10 
p = [0.2 0.2 0.4 0.4 0.7 0.7 0.2 0.2 0.8 0.8 0.5 0.5 0.2 0.2]
len_p = length(p)
max_stock = 20 
min_stock = 1
max_order = 5

5

- policy estimation code

In [37]:
function get_cost_of_complete_policy(policy, n, N=10000)
    costs = zeros(N)
    for itr ∈ 1:N
        stock = initial_stock
        cost = 0
        for t ∈ 1:T
            up_stock = min(policy[t, stock], 5, max_stock - stock)
            cost -= up_stock * price_up 
            stock += up_stock
            d = rand(Binomial(n, p[t]))
            down_stock = min(d, stock - 1) 
            stock -= down_stock
            cost += down_stock * price_down  
            cost -= storage_cost * stock  
        end
        costs[itr] = cost
    end
    std_var = std(costs)
    final_cost = mean(costs)
    CI_half_lenght = std_var * 1.96 / sqrt(N)
    return final_cost, final_cost + CI_half_lenght, final_cost - CI_half_lenght
end

get_cost_of_complete_policy (generic function with 2 methods)

In [38]:
function get_cost_of_policy(policy, n, N=10000)
    costs = zeros(N)
    for itr ∈ 1:N
        stock = initial_stock  
        cost = 0.0
        for t ∈ 1:T  
            up_stock = min(policy[t], 5, max_stock - stock)
            cost -= up_stock * price_up 
            stock += up_stock
            d = rand(Binomial(n, p[t]))  
            down_stock = min(d, stock - 1)  
            stock -= down_stock
            cost += down_stock * price_down  
            cost -= storage_cost * stock 
        end
        costs[itr] = cost
    end
    final_cost = mean(costs)
    CI_half_length = 1.96 * std(costs) / sqrt(N)
    return (final_cost, final_cost + CI_half_length, final_cost - CI_half_length)
end

get_cost_of_policy (generic function with 2 methods)

In [39]:
test_policy = 5 * ones(T)
cost,CI_upper_bound,CI_lower_bound = get_cost_of_policy(test_policy,10)
print("the expected cost obtained is $cost and a 95% confidence interval is [$CI_lower_bound,$CI_upper_bound]")

the expected cost obtained is 98.17491000000001 and a 95% confidence interval is [97.88445577012264,98.46536422987738]

- dynamic programming

In [40]:
function inventory_dp()
    V = zeros(T+1, max_stock + 1)
    policy = zeros(Int, T, max_stock + 1) 

    binom_probs = Vector{Vector{Float64}}(undef, T)
    for t in 1:T
        dist = Binomial(10, p[t])
        binom_probs[t] = [pdf(dist, k) for k in 0:10]
    end

    for t in T:-1:1  
        probs = binom_probs[t]
        
        for x in 0:max_stock  
            max_expected_cost = -Inf
            best_u = 0
            for u in 0:min(max_order, max_stock - x)
                S = x + u  
                total_cost = 0.0
                for (k_idx, k) in enumerate(0:10)
                    prob = probs[k_idx]
                    sales = min(k, S-1)
                    next_stock = max(0, S - k)
                    order_cost = price_up*u
                    holding_cost = storage_cost * next_stock
                    revenue = price_down* sales
                    cost_component = revenue - order_cost - holding_cost
                    if t < T
                        cost_component += V[t+1, next_stock + 1] 
                    end
                    
                    total_cost += prob * cost_component
                end
                if total_cost > max_expected_cost
                    max_expected_cost = total_cost
                    best_u = u
                end
            end
            V[t, x+1] = max_expected_cost
            policy[t, x+1] = best_u
        end
    end

    return V, policy
end

V, policy = inventory_dp()

expected_cost = V[1, 10+1]

116.75121513292399

In [41]:
cost,LB,UB = get_cost_of_complete_policy(policy,10)
print("the expected cost obtained for the optimal policy is $cost and a 95% confidence interval is [$UB,$LB]")

the expected cost obtained for the optimal policy is 116.42698999999999 and a 95% confidence interval is [116.20892435238058,116.6450556476194]

In [66]:
function inventory_dp_with_time(T=96)
    V = zeros(T+1, max_stock + 1)
    policy = zeros(Int, T, max_stock + 1) 

    binom_probs = Vector{Vector{Float64}}(undef, T)
    for t in 1:T
        
        dist = Binomial(10, p[(t-1)%len_p + 1])
        binom_probs[t] = [pdf(dist, k) for k in 0:10]
    end

    for t in T:-1:1  
        probs = binom_probs[t]
        
        for x in 0:max_stock  
            max_expected_cost = -Inf
            best_u = 0
            for u in 0:min(max_order, max_stock - x)
                S = x + u  
                total_cost = 0.0
                for (k_idx, k) in enumerate(0:10)
                    prob = probs[k_idx]
                    sales = min(k, S-1)
                    next_stock = max(0, S - k)
                    order_cost = price_up*u
                    holding_cost = storage_cost * next_stock
                    revenue = price_down* sales
                    cost_component = revenue - order_cost - holding_cost
                    if t < T
                        cost_component += V[t+1, next_stock + 1] 
                    end
                    
                    total_cost += prob * cost_component
                end
                if total_cost > max_expected_cost
                    max_expected_cost = total_cost
                    best_u = u
                end
            end
            V[t, x+1] = max_expected_cost
            policy[t, x+1] = best_u
        end
    end

    return V, policy
end

V, policy = inventory_dp_with_time()

expected_cost = V[1, 10+1]

765.2803067250243

In [70]:
function solve_stock_dp()
    T = 14
    max_stock = 20
    min_stock = 1
    max_order = 5
    p = [0.2, 0.2, 0.4, 0.4, 0.7, 0.7, 0.2, 0.2, 0.8, 0.8, 0.5, 0.5, 0.2, 0.2]
    
    
    stock_range = min_stock:max_stock
    order_range = 0:max_order
    
    
    V = Array{Float64}(undef, T+1, length(stock_range), length(order_range), length(order_range))
    policy = Array{Int}(undef, T, length(stock_range), length(order_range), length(order_range))
    
    
    fill!(view(V, T+1, :, :, :), 0.0)

    
    binom_probs = Vector{Vector{Float64}}(undef, T)
    for t in 1:T
        dist = Binomial(10, p[t])
        binom_probs[t] = [pdf(dist, d) for d in 0:10]
    end

    for t in T:-1:1
        for x in stock_range  
            for u1 in order_range 
                for u2 in order_range  
                    min_cost = Inf
                    best_action = 0

                    
                    current_stock = x + u1
                    current_stock = min(current_stock, max_stock)

                    
                    expected_cost = 0.0
                    for (d_idx, d) in enumerate(0:10)
                        prob = binom_probs[t][d_idx]
                        
                       
                        after_demand = max(min_stock, current_stock - d)
                        sales = min(d, current_stock)
                        
                        
                        holding = 0.1 * after_demand
                        revenue = 3 * sales
                        
                        
                        best_u = 0
                        min_cost_d = Inf
                        
                        
                        max_feasible = min(max_order, max_stock - after_demand)
                        
                        for u in 0:max_feasible
                            
                            next_stock = max(min_stock, after_demand + u2)  
                            next_stock = min(next_stock, max_stock)
                            

                            x_idx = next_stock - min_stock + 1
                            u1_idx = u2 + 1  
                            u2_idx = u + 1
                            
                            order_cost = u
                            total_cost = order_cost + holding - revenue
                            
                            if t < T
                                total_cost += V[t+1, x_idx, u1_idx, u2_idx]
                            end
                            
                            if total_cost < min_cost_d
                                min_cost_d = total_cost
                                best_u = u
                            end
                        end
                        
                        expected_cost += prob * min_cost_d
                    end
                    
                    x_idx = x - min_stock + 1
                    u1_idx = u1 + 1
                    u2_idx = u2 + 1
                    
                    V[t, x_idx, u1_idx, u2_idx] = expected_cost
                    policy[t, x_idx, u1_idx, u2_idx] = best_action
                end
            end
        end
    end

    
    initial_x_idx = 10 - min_stock + 1
    return -V[1, initial_x_idx, 1, 1]  
end

# Run the DP solution
optimal_value = solve_stock_dp()
println("Optimal expected value: ", round(optimal_value, digits=2))

Optimal expected value: 148.0


# 3 - Dice trading

- constants :

In [43]:
T = 10
price_per_dice = 5
max_dice = 3

3

In [44]:
function simulate_strategy(strategy,N=10000)
    results = zeros(N)
    
    for i in 1:N
        x = 0  
        d = 1  
        for t in 1:T
            action = strategy[t, x+1, d]
            
            if action == 1 && x >= 6 && d < 3
                x -= price_per_dice
                d += 1
            end
            
            roll = rand(1:6, d)
            x += maximum(roll)
        end
        results[i] = x
    end
    
    μ = mean(results)
    ci = 1.96 * std(results)/sqrt(N)
    return μ, μ - ci, μ + ci
end

simulate_strategy (generic function with 2 methods)

In [45]:
function solve_dice_dp()
    T = 10
    max_dice = 3
    max_points = 61 # 6 * 10  and we add 1 for safety 

    V = zeros(T+1, max_points+1, max_dice+1)
    policy = zeros(Int, T, max_points+1, max_dice+1)

    for x in 0:max_points, d in 1:max_dice
        V[T+1, x+1, d] = x
    end
    

    dice_probs = [Dict{Int,Float64}() for _ in 1:3]
    for dice in 1:3
        for m in 1:6
            prob = (m/6)^dice - ((m-1)/6)^dice
            dice_probs[dice][m] = prob
        end
    end
    
    for t in T:-1:1
        for x in 0:max_points
            for d in 1:max_dice
                best_value = -Inf
                best_action = 0

                for action in 0:1
                    if action == 1 && (x < 6 || d >= 3)
                        continue
                    end
                    
                    current_value = 0.0
                    if action == 1
                        new_d = min(d + 1, 3)
                        new_x = x - price_per_dice
                    else
                        new_d = d
                        new_x = x
                    end
                    
                    new_x < 0 && continue
                    
                    for (m, prob) in dice_probs[new_d]
                        next_x = new_x + m
                        next_x = min(next_x, max_points)
                        
                        if t+1 <= T && next_x <= max_points
                            current_value += prob * V[t+1, next_x+1, new_d]
                        else
                            current_value += prob * next_x
                        end
                    end
                    
                    if current_value > best_value
                        best_value = current_value
                        best_action = action
                    end
                end
                
                V[t, x+1, d] = best_value
                policy[t, x+1, d] = best_action
            end
        end
    end
    
    V, policy
end

solve_dice_dp (generic function with 1 method)

In [46]:
V, policy = solve_dice_dp()
print("the gains from the optimal strategy is $(V[1,1,1])")
println()
μ,LB,UB =  simulate_strategy(policy)
print("through simulation we find the gain associated with the optimal strategy is $μ and the 95% confidence interval is [$LB,$UB]")

the gains from the optimal strategy is 37.62152777777777
through simulation we find the gain associated with the optimal strategy is 37.564 and the 95% confidence interval is [37.46421175118231,37.66378824881769]

In [47]:
policy

10×62×4 Array{Int64, 3}:
[:, :, 1] =
 0  0  0  0  0  0  1  1  1  1  1  1  1  …  1  0  0  0  0  1  1  1  1  1  1  1
 0  0  0  0  0  0  1  1  1  1  1  1  1     1  1  0  0  0  0  0  1  1  1  1  1
 0  0  0  0  0  0  1  1  1  1  1  1  1     1  1  1  0  0  0  0  0  1  1  1  1
 0  0  0  0  0  0  1  1  1  1  1  1  1     1  1  1  1  0  0  0  0  0  1  1  1
 0  0  0  0  0  0  1  1  1  1  1  1  1     1  1  1  1  1  0  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  1  1  1  1  1  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 2] =
 0  0  0  0  0  0  0  0  0  0  0  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0

In [48]:
function precompute_max_probs()
    max_probs = Dict{Int,Dict{Int,Float64}}()
    for dice in 1:MAX_DICE
        probs = Dict{Int,Float64}()
        for m in 1:6
            prob = (m/6)^dice - ((m-1)/6)^dice
            probs[m] = prob
        end
        max_probs[dice] = probs
    end
    return max_probs
end

precompute_max_probs (generic function with 1 method)

In [49]:
function solve_dp_spend(total_turns=10, max_dice=5) 
    max_points = 12 * total_turns  

    value_grid = zeros(Float64, total_turns+1, max_points+1, max_dice+1)

    for points ∈ 0:max_points, dice ∈ 1:max_dice
        value_grid[total_turns+1, points+1, dice] = points
    end

    probability_matrix = Vector{Vector{Float64}}(undef, max_dice)
    for dice ∈ 1:max_dice
        probabilities = Float64[]
        for roll ∈ 1:6
            push!(probabilities, (roll/6)^dice - ((roll-1)/6)^dice)
        end
        probability_matrix[dice] = probabilities
    end

    for current_turn ∈ total_turns:-1:1  
        for dice ∈ 1:max_dice, points ∈  0:max_points  
            optimal_value = -Inf 
            
            for action ∈ [0, 1]  
                if action == 1 && (points < 6 || dice == max_dice)
                    continue
                end

                post_action_points = points - 5 * action
                post_action_dice = dice + action

                expected_return = 0.0

                for die_roll ∈ 1:6  
                    roll_prob = probability_matrix[post_action_dice][die_roll]
                    updated_points_no_spend = min(post_action_points + die_roll, max_points)
                    value_no_spend = value_grid[current_turn+1, updated_points_no_spend+1, post_action_dice]

                    if post_action_dice >= 2
                        updated_points_spend = min(post_action_points + 2*die_roll, max_points)
                        value_spend = value_grid[current_turn+1, updated_points_spend+1, post_action_dice-1]
                    else
                        value_spend = -Inf
                    end
                    expected_return += roll_prob * max(value_no_spend, value_spend)
                end
                optimal_value = max(optimal_value, expected_return)
            end

            value_grid[current_turn, points+1, dice] = optimal_value
        end
    end

    return value_grid
end

solve_dp_spend (generic function with 3 methods)

In [50]:
print("the optimal value is $(solve_dp_spend(10,5)[1,1,1])")

In [51]:
matrix = solve_dp_spend(10,5)[:,:,1]

the optimal value is 47.59380284322565

11×121 Matrix{Float64}:
 47.5938   49.3069   50.956    52.5375  …  120.0    120.0    120.0  120.0
 41.8268   43.5078   45.1285   46.6852     120.0    120.0    120.0  120.0
 36.1867   37.8247   39.4072   40.9307     120.0    120.0    120.0  120.0
 30.7175   32.2995   33.8306   35.3081     120.0    120.0    120.0  120.0
 25.4418   26.9649   28.4402   29.8664     120.0    120.0    120.0  120.0
 20.3778   21.8306   23.2419   24.6119  …  120.0    120.0    120.0  120.0
 15.5655   16.943    18.2822   19.5836     120.0    120.0    120.0  120.0
 11.034    12.2963   13.5463   14.784      120.0    120.0    120.0  120.0
  7.07407   8.14815   9.22222  10.2963     119.972  120.0    120.0  120.0
  3.5       4.5       5.5       6.5        119.5    119.833  120.0  120.0
  0.0       1.0       2.0       3.0     …  117.0    118.0    119.0  120.0

In [52]:
function solve_dp_spend_with_selling(total_turns=10, max_dice=5) 
    K = [0,2,4,5,8]
    max_points = 12 * total_turns  

    value_grid = zeros(Float64, total_turns+1, max_points+1, max_dice+1)

    for points ∈ 0:max_points, dice ∈ 1:max_dice
        value_grid[total_turns+1, points+1, dice] = points + K[dice]
    end

    probability_matrix = Vector{Vector{Float64}}(undef, max_dice)
    for dice ∈ 1:max_dice
        probabilities = Float64[]
        for roll ∈ 1:6
            push!(probabilities, (roll/6)^dice - ((roll-1)/6)^dice)
        end
        probability_matrix[dice] = probabilities
    end

    for current_turn ∈ total_turns:-1:1  
        for dice ∈ 1:max_dice, points ∈ 0:max_points  
            optimal_value = -Inf 
            
            for action ∈ [0, 1]  
                if action == 1 && (points < 6 || dice == max_dice)
                    continue
                end

                post_action_points = points - 5 * action
                post_action_dice = dice + action

                expected_return = 0.0

                for die_roll ∈ 1:6  
                    roll_prob = probability_matrix[post_action_dice][die_roll]
                    updated_points_no_spend = min(post_action_points + die_roll, max_points)
                    value_no_spend = value_grid[current_turn+1, updated_points_no_spend+1, post_action_dice]

                    if post_action_dice >= 2
                        updated_points_spend = min(post_action_points + 2*die_roll, max_points)
                        value_spend = value_grid[current_turn+1, updated_points_spend+1, post_action_dice-1]
                    else
                        value_spend = -Inf
                    end
                    expected_return += roll_prob * max(value_no_spend, value_spend)
                end
                optimal_value = max(optimal_value, expected_return)
            end

            value_grid[current_turn, points+1, dice] = optimal_value
        end
    end

    return value_grid
end

solve_dp_spend_with_selling (generic function with 3 methods)

In [53]:
matrix = solve_dp_spend_with_selling(10,5)

11×121×6 Array{Float64, 3}:
[:, :, 1] =
 47.7127  49.4287   51.0804   52.6641  …  128.0    128.0    128.0    128.0
 41.9341  43.619    45.2432   46.8029     128.0    128.0    128.0    128.0
 36.2789  37.9219   39.509    41.0365     128.0    128.0    128.0    128.0
 30.7908  32.3785   33.9152   35.398      128.0    128.0    128.0    128.0
 25.4996  27.0268   28.5062   29.9364     127.998  127.999  128.0    128.0
 20.4213  21.8789   23.2946   24.6684  …  127.844  127.923  127.965  127.985
 15.5936  16.9749   18.3183   19.6243     125.728  126.283  126.777  127.116
 11.054   12.3194   13.5718   14.811      123.76   124.041  124.284  124.448
  7.0787   8.15741   9.23611  10.3148     121.667  122.185  122.699  123.065
  3.5      4.5       5.5       6.5        119.5    119.833  120.611  121.167
  0.0      1.0       2.0       3.0     …  117.0    118.0    119.0    120.0

[:, :, 2] =
 57.1068   58.5241   59.8618  61.1113  …  128.0    128.0    128.0    128.0
 51.1908   52.5921   53.9152  55.1533